In [6]:
import c3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import os
from read_mocap import PATH_TO_DATA, PATH_TO_RECORDED_DATA, read_and_preprocess_csv_data
# PATH_TO_DATA = 'Data'
# PATH_TO_RECORDED_DATA = os.path.join(PATH_TO_DATA, 'human arm tracks')
print(f'path to recorded data = {PATH_TO_RECORDED_DATA}')
# Get the csv records only
records = [f for f in sorted(os.listdir(PATH_TO_RECORDED_DATA)) if f.endswith('.csv')]
print(f'recorded list = {records}')



data = read_and_preprocess_csv_data(os.path.join(PATH_TO_RECORDED_DATA,records[-1]), 0, 9)

print(data.columns)
display(data)

path to recorded data = Data/human arm tracks
recorded list = ['Take 2022-12-14 05.13.01 PM_001.csv', 'Take 2023-01-18 07.26.05 PM_001.csv', 'Take 2023-01-18 3.csv', 'Take 2023-01-18 4.csv', 'Take 2023-01-18 5.csv', 'Take 2023-01-18 6.csv', 'Take 2023-01-18 7.csv', 'Take 2023-01-18 8.csv', 'Take 2023-01-19 1.csv', 'Take 2023-01-19 2 processed.csv', 'Take 2023-01-19 3 processed.csv', 'Take 2023-01-19 4 processed .csv']
Index(['Frame', 'Time (Seconds)', 'PIH_X_Rotation', 'PIH_Y_Rotation',
       'PIH_Z_Rotation', 'PIH_W_Rotation', 'PIH_X_Position', 'PIH_Y_Position',
       'PIH_Z_Position'],
      dtype='object')


,Frame,Time (Seconds),PIH_X_Rotation,PIH_Y_Rotation,PIH_Z_Rotation,PIH_W_Rotation,PIH_X_Position,PIH_Y_Position,PIH_Z_Position
0,0,0.00,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389
1,1,0.01,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389
2,2,0.02,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389
3,3,0.03,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389
4,4,0.04,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389
...,...,...,...,...,...,...,...,...,...
7504,7504,75.04,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764
7505,7505,75.05,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764
7506,7506,75.06,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764
7507,7507,75.07,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764


### Check if there are any NaNs left after interpolation 

In [ ]:
Ndata_idxs = data.notnull()
Ndata = data[Ndata_idxs]
# display(Ndata)
sim = Ndata == data
print('Data frame where Nans exist:\n')
display(sim[sim.eq(False).any(1)])
print('Data Frame of comparing similarity\n')
display(sim)

In [ ]:
del data